# Default PowerSploit and Empire Schtasks Persistence
Detects the creation of a schtask via PowerSploit or Empire Default Configuration.

## Rule Content
```
- title: Default PowerSploit and Empire Schtasks Persistence
  id: 56c217c3-2de2-479b-990f-5c109ba8458f
  status: experimental
  description: Detects the creation of a schtask via PowerSploit or Empire Default
    Configuration.
  references:
  - https://github.com/0xdeadbeefJERKY/PowerSploit/blob/8690399ef70d2cad10213575ac67e8fa90ddf7c3/Persistence/Persistence.psm1
  - https://github.com/EmpireProject/Empire/blob/master/lib/modules/powershell/persistence/userland/schtasks.py
  - https://github.com/EmpireProject/Empire/blob/master/lib/modules/powershell/persistence/elevated/schtasks.py
  author: Markus Neis, @Karneades
  date: 2018/03/06
  logsource:
    product: windows
    category: process_creation
    service: null
  detection:
    selection:
      ParentImage:
      - '*\powershell.exe'
      CommandLine:
      - '*schtasks*/Create*/SC *ONLOGON*/TN *Updater*/TR *powershell*'
      - '*schtasks*/Create*/SC *DAILY*/TN *Updater*/TR *powershell*'
      - '*schtasks*/Create*/SC *ONIDLE*/TN *Updater*/TR *powershell*'
      - '*schtasks*/Create*/SC *Updater*/TN *Updater*/TR *powershell*'
    condition: selection
  tags:
  - attack.execution
  - attack.persistence
  - attack.privilege_escalation
  - attack.t1053
  - attack.t1086
  - attack.s0111
  - attack.g0022
  - attack.g0060
  - car.2013-08-001
  falsepositives:
  - False positives are possible, depends on organisation and processes
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_parent_path.keyword:(*\\powershell.exe) AND process_command_line.keyword:(*schtasks*\/Create*\/SC\ *ONLOGON*\/TN\ *Updater*\/TR\ *powershell* OR *schtasks*\/Create*\/SC\ *DAILY*\/TN\ *Updater*\/TR\ *powershell* OR *schtasks*\/Create*\/SC\ *ONIDLE*\/TN\ *Updater*\/TR\ *powershell* OR *schtasks*\/Create*\/SC\ *Updater*\/TN\ *Updater*\/TR\ *powershell*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()